# Task 1 Descriptive Statistics 

## Import and Initializing

In [1]:
import itertools

import numpy as np  # pip install numpy
import scipy.stats  # pip install scipy

import pandas as pd  # pip install pandas
import matplotlib.pyplot as plt  # pip install matplotlib
import seaborn as sns  # pip install seaborn

sns.set_context("notebook")
sns.set_style("whitegrid")

FIG_SIZE = (14, 4)

In [2]:
def ci_95(data):
    """https://stackoverflow.com/a/15034143"""
    confidence = 0.95
    n = len(data)
    std_err_of_measurement = scipy.stats.sem(data)
    ci = std_err_of_measurement * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return ci

## Loading the data

In [3]:
sheets = pd.read_excel("../datasets/DB04_speech_quality_crowdsourcing_dataset.xlsx", sheet_name=None)
print("Sheets:")
list(sheets.keys())

Sheets:


['Description',
 'CS -Full',
 'CS-per file',
 'CS-per condition',
 'CS participants',
 'Lab-per condition']

In [4]:
prev_max_cols = pd.get_option('display.max_columns')
pd.set_option('display.max_columns', None)  
display(sheets["CS-per file"].head(3))
pd.set_option('display.max_columns', prev_max_cols)  

,files,Num_Ratings,condition,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,MOS,STD
0,D401_c01_ef01_s001.wav,9,1.0,5,4,5,5,5,5,5.0,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.77778,0.440959
1,D401_c01_ef01_s002.wav,9,1.0,4,5,5,5,5,5,4.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.77778,0.440959
2,D401_c01_ef01_s003.wav,10,1.0,5,5,4,5,5,5,5.0,5.0,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.8,0.421637


## Data preparation

In [5]:
sheet = sheets["CS-per file"]

def parse_condition(file_name):
    _, cond, _, _ = file_name.split("_")
    return cond

sheet["condition_filled"] = sheet.files.transform(parse_condition)

In [6]:
persons = [col for col in sheet.columns if col.startswith("P")]
columns = ["condition_filled"] + persons
all_conditions = sheet[columns]
all_conditions

,condition_filled,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18
0,c01,5,4,5,5,5,5,5.0,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c01,4,5,5,5,5,5,4.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c01,5,5,4,5,5,5,5.0,5.0,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,c01,5,5,5,5,4,5,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,c01,4,4,5,5,5,5,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147,c48,4,2,4,5,3,5,4.0,4.0,3.0,5.0,3.0,4.0,3.0,5.0,4.0,1.0,2.0,4.0
1148,c48,2,4,3,3,3,4,3.0,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1149,c48,4,4,3,3,3,4,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1150,c48,5,4,4,4,4,4,4.0,4.0,3.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Calculating MOS, std and 95% CI

In [7]:
by_condition = all_conditions.groupby("condition_filled")

def block_stats(data, columns, agg_fns, agg_names):
    selected = data[columns].to_numpy()
    selected = selected[~np.isnan(selected)]
    
    results = [fn(selected) for fn in agg_fns]
    return pd.Series(results, index=agg_names)

stats_names = ["mean", "std", "95% CI", "n"]
stats_fns = [np.mean, np.std, ci_95, len]

statistics = by_condition.apply(
    block_stats,
    columns=persons,
    agg_fns=stats_fns,
    agg_names=stats_names,
)

In [8]:
pre_calculated = sheets["CS-per condition"].set_index("condition")
pd.concat([statistics, pre_calculated], axis="columns", keys=["self calculated", "pre calculated"], sort=True)

self calculated                            pre calculated           \
               mean       std    95% CI      n            MOS      STD   
c01        4.836449  0.428406  0.057861  214.0        4.83645  0.42941   
c02        1.541284  0.583128  0.078021  218.0              ?        ?   
c03        4.351852  0.802944  0.107936  216.0            NaN      NaN   
c04        2.783410  0.912838  0.122421  217.0            NaN      NaN   
c05        3.537037  0.731892  0.098385  216.0            NaN      NaN   
c06        4.623256  0.547653  0.073792  215.0            NaN      NaN   
c07        3.665094  1.035278  0.140495  212.0            NaN      NaN   
c08        3.933962  0.723838  0.098230  212.0            NaN      NaN   
c09        3.117371  0.855981  0.115886  213.0            NaN      NaN   
c10        4.518182  0.642889  0.085619  220.0            NaN      NaN   
c11        4.246637  0.779315  0.103076  223.0            NaN      NaN   
c12        1.504717  0.710419  0.096409  212.0            NaN      NaN   
c13        4.464455  0.625233  0.085053  211.0            NaN      NaN   
c14        2.779279  0.905898  0.120093  222.0            NaN      NaN   
c15        3.540909  1.071600  0.142714  220.0            NaN      NaN   
c16        3.680365  0.792734  0.105819  219.0            NaN      NaN   
c17        4.229665  0.950890  0.129981  209.0            NaN      NaN   
c18        3.077982  1.026427  0.137333  218.0            NaN      NaN   
c19        4.509524  0.619066  0.084418  210.0            NaN      NaN   
c20        2.787611  0.856535  0.112524  226.0            NaN      NaN   
c21        3.027523  0.834560  0.111662  218.0            NaN      NaN   
c22        2.867925  0.819290  0.111184  212.0            NaN      NaN   
c23        3.732394  1.001751  0.135621  213.0            NaN      NaN   
c24        1.497738  0.690101  0.091695  221.0            NaN      NaN   
c25        4.299517  0.866527  0.119030  207.0            NaN      NaN   
c26        1.506912  0.637679  0.085519  217.0            NaN      NaN   
c27        2.730769  0.977937  0.134005  208.0            NaN      NaN   
c28        2.327354  0.817694  0.108153  223.0            NaN      NaN   
c29        2.953917  0.944397  0.126653  217.0            NaN      NaN   
c30        1.954751  0.741612  0.098539  221.0            NaN      NaN   
c31        3.128440  0.894437  0.119673  218.0            NaN      NaN   
c32        2.814815  0.856869  0.115185  216.0            NaN      NaN   
c33        3.361991  1.004606  0.133484  221.0            NaN      NaN   
c34        3.319444  0.852443  0.114590  216.0            NaN      NaN   
c35        2.714953  0.836135  0.112930  214.0            NaN      NaN   
c36        2.442922  1.006909  0.134409  219.0            NaN      NaN   
c37        3.307692  0.826805  0.109859  221.0            NaN      NaN   
c38        2.077626  0.900736  0.120236  219.0            NaN      NaN   
c39        3.074074  0.689924  0.092743  216.0            NaN      NaN   
c40        3.292793  0.747273  0.099064  222.0            NaN      NaN   
c41        2.146919  0.914459  0.124398  211.0            NaN      NaN   
c42        2.516588  0.804878  0.109491  211.0            NaN      NaN   
c43        4.531532  0.661963  0.087755  222.0            NaN      NaN   
c44        4.675799  0.565289  0.075458  219.0            NaN      NaN   
c45        3.859729  0.957588  0.127236  221.0            NaN      NaN   
c46        3.734234  1.002968  0.132961  222.0            NaN      NaN   
c47        4.583333  0.619198  0.080982  228.0            NaN      NaN   
c48        3.665179  0.834224  0.110088  224.0            NaN      NaN   

                     
          CI95    n  
c01  0.0484937  214  
c02          ?    ?  
c03        NaN  NaN  
c04        NaN  NaN  
c05        NaN  NaN  
c06        NaN  NaN  
c07        NaN  NaN  
c08        NaN  NaN  
c09        NaN  NaN  
c10        NaN  NaN  
c11        NaN  NaN  
c12        NaN  N